In [1]:
%env HSA_OVERRIDE_GFX_VERSION=10.3.0
%env ROCM_PATH=/opt/rocm
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["AUTOGRAPH_VERBOSITY"] = "0"

env: HSA_OVERRIDE_GFX_VERSION=10.3.0
env: ROCM_PATH=/opt/rocm


In [2]:
from IPython.display import display
from ipyfilechooser import FileChooser
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

use_video_in_configuration =  int(configParser.get('fineTuneStableDiffusionTesting', 
                                                   'use_video_in_configuration'))

video_path =  configParser.get('fineTuneStableDiffusionTesting', 
                                                   'video_path')

ttwbdf =  int(configParser.get('fineTuneStableDiffusionTesting', 'time_to_wait_before_deleting_files'))


dev_mode =  int(configParser.get('fineTuneStableDiffusionTesting', 'dev_mode'))

cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))


if(use_video_in_configuration == 0):
    fc = FileChooser('/')
    


    # Create and display a FileChooser widget

    # Set a file filter patern
    fc.filter_pattern = '*.mp4'
    display(fc)

    # Print the selected filename
    print(fc.selected_filename)


In [3]:
if(use_video_in_configuration == 0):
    video_path = fc.selected

In [4]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [5]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [6]:
print("Number of cpus to use for multiprocessing : ", cpus)
if(cuda == 0):
    print("Not using cuda")
else:
    print("Will try to use cuda, if no cuda is present please set cuda = 0 in configuration.txt")

Number of cpus to use for multiprocessing :  8
Will try to use cuda, if no cuda is present please set cuda = 0 in configuration.txt


In [7]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [8]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
audio_length_used =  configParser.get('train_imagen', 'audio_length_used') 
model_filename =  configParser.get('train_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('extractOpenL3', 'openl3_mode')

In [9]:
dataGotten = [['1',video_path,'50','black','man',[],[],'English',[]]]
df = pd.DataFrame(dataGotten,columns = ['ID','VIDEO_PATH','AGE','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','caption_l','image_path'])


In [10]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [11]:

import torchaudio
import speechbrain as sb
from speechbrain.pretrained import EncoderClassifier
import openl3
import openl3
import soundfile as sf
import torchaudio
import speechbrain as sb
from speechbrain.pretrained import EncoderClassifier


model = openl3.models.load_audio_embedding_model(input_repr="mel128", content_type="music",
                                                embedding_size=512)


classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb")



from pydub import AudioSegment
import math
# Get original duration of video
audio = AudioSegment.from_file(video_path)
audio_length_og = math.floor(audio.duration_seconds)
audio_length_og

video_filename = os.path.basename(video_path)

absPathAudio_w = os.path.abspath(audio_folder) + "/" + video_filename
absPathAudio = os.path.splitext(absPathAudio_w)[0]+'_audio.wav'
absPathAudio_w = os.path.splitext(absPathAudio_w)[0]


import subprocess
# Extract audio monochannel and with 16khz and put it in absPathAudio

command = "ffmpeg -nostats -loglevel 0 -y -i '" + video_path + "' -acodec pcm_s16le -ab 160k -ac 1 -ar 16000 -vn '" + absPathAudio  + "'"
subprocess.call(command, shell=True)

import torchaudio
from threading import Thread
import time



# Function to delete audio temp files
def delFiles(filesToDelete):
    time.sleep(ttwbdf)  # wait a bit
    for file in filesToDelete:  
        try:
            os.remove(file)
        except OSError:
            pass


# Will either truncate or loop the original video to reach audio_length (6,12 or 24)
audio_length_list = [24]
for audio_length in audio_length_list:
    path_var_len_audio =  absPathAudio_w + "audio" + str(audio_length) + "s.wav"    # path to the variable length audio
    path_var_len_audio_temp =  absPathAudio_w + "audio_temp" + str(audio_length) + "s.wav"  # path to a temp version of the variable length audio

    if(audio_length_og > audio_length):
        # Truncate    

        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + absPathAudio + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)


    else:
        # Loop then truncaate
        #print("lesa")
        twoDigitLenStr = f"{audio_length:02}"
        #print(twoDigitLenStr)
        command = "ffmpeg -nostats -loglevel 0 -y -stream_loop -1 -i '" + absPathAudio + "' -t \"00:00:"+twoDigitLenStr+".000\" -codec:a \"aac\" -f \"wav\" -c copy '"+ path_var_len_audio_temp + "'"
        subprocess.call(command, shell=True)
        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + path_var_len_audio_temp + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)

    # Extract speaker embeddings
    signal, fs = torchaudio.load(path_var_len_audio)
    embeddings = classifier.encode_batch(signal)
    embeddingsPickle = pickle.dumps(embeddings.cpu().detach().numpy())

    audio, sr = sf.read(path_var_len_audio)

    hop_size = -1
    if(openl3_mode == 'imagen'):
        hop_size = audio_length/250
    elif(openl3_mode == 'stable'):
        hop_size = 24/50
    else:
        raise ValueError('openl3_mode in configuration must either be stable or imagen') 
    
    emb, ts = openl3.get_audio_embedding(audio, sr,hop_size=hop_size,verbose=0,model=model)

    embeddingsPickle2 = pickle.dumps(emb)

    df['SPEAKER_EMB'] = embeddingsPickle
    df['AUDIO_EMB'] = embeddingsPickle2

    

    ftd = [absPathAudio,path_var_len_audio,os.path.basename(path_var_len_audio),path_var_len_audio_temp]
    tDelete = Thread(target=delFiles, args=(ftd,))   # spawn a process
    tDelete.start()



/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")
/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/site-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [12]:
df3 = df[["image_path","caption_e","caption_g","caption_l"]]
data_frame = df3
data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
data_frame['AUDIO_EMB'] = df['AUDIO_EMB']

In [13]:
def speaker_emb_preprocess(speaker_emb2):

    speaker_emb2 = pickle.loads(speaker_emb2)
    speaker_emb2 = speaker_emb2.squeeze()
    speaker_emb2 = np.pad(speaker_emb2, (288), 'constant', constant_values=(0))
    speaker_emb2 = np.tile(speaker_emb2, (1, 1))
    speaker_emb2 = speaker_emb2 / 200.0
    speaker_emb2 = np.array(speaker_emb2).tolist()
    return speaker_emb2


def audio_emb_preprocess2(speaker_emb2):
    speaker_emb2 = pickle.loads(speaker_emb2)
    a = speaker_emb2 #np.zeros(shape=(24, 512))
    b = np.zeros(shape=(a.shape[0], 768-a.shape[1]))
    c = np.zeros(shape=(250-a.shape[0], 768))
    arr = np.concatenate((a, b), axis=1)
    arr = np.concatenate((arr, c), axis=0)
    arr = arr / 10.0
    speaker_emb2 = np.array(arr).tolist()
    return speaker_emb2

from PIL import Image
def getImage(face_path):
    im = Image.open(face_path)
    im.load() # required for png.split()

    im2 = Image.new("RGB", im.size, (255, 255, 255))
    im2.paste(im, mask=im.split()[3]) # 3 is the alpha channel
    im3 = np.array(im2)
    im4 = np.rollaxis(im3,2)
    return im4

In [14]:
import random
def process_gender(gender):
    if(gender == 'man'):
        return np.ones(768)
    elif(gender == "woman"):
        return np.ones(768) * -1
    else:
        return np.zeros(768)
    
# TODO
def process_ethnicity(eth):
    x = np.zeros(768)
    if(eth == "indian"):
        x[0] = 1
    elif(eth == "asian"):
        x[16]=1
    elif(eth == "latino hispanic"):
        x[2]=1
    elif(eth == "black"):
        x[3]=1
    elif(eth == "middle eastern"):
        x[4]=1
    elif(eth == "white"):
        x[5]=1 
    else:
        x = x
    return x     

def process_language(lan):
    x = np.zeros(768)
    if(lan == "Arabic"):
        x[0] = 1
    elif(lan == "Portuguese"):
        x[16]=1
    elif(lan == "Romansh_Sursilvan"):
        x[2]=1
    elif(lan == "Japanese"):
        x[3]=1
    elif(lan == "Ukranian"):
        x[4]=1
    elif(lan == "German"):
        x[5]=1   
    elif(lan == "Chinese_China"):
        x[6]=1   
    elif(lan == "Welsh"):
        x[7]=1  
    elif(lan == "English"):
        x[8]=1
    elif(lan == "Kabyle"):
        x[9]=1 
    elif(lan == "Kyrgyz"):
        x[10]=1
    elif(lan == "Georgian"):
        x[11]=1
    elif(lan == "Persian"):
        x[12]=1 
    elif(lan == "French"):
        x[13]=1
    elif(lan == "Interlingua"):
        x[14]=1
    elif(lan == "Swedish"):
        x[15]=1
    elif(lan == "Spanish"):
        x[16]=1 
    elif(lan == "Dhivehi"):
        x[17]=1
    elif(lan == "Kinyarwanda"):
        x[18]=1 
    elif(lan == "Tatar"):
        x[19]=1
    elif(lan == "Hakha_Chin"):
        x[20]=1 
    elif(lan == "Tamil"):
        x[21]=1 
    elif(lan == "Greek"):
        x[22]=1
    elif(lan == "Latvian"):
        x[23]=1 
    elif(lan == "Russian"):
        x[24]=1
    elif(lan == "Breton"):
        x[25]=1
    elif(lan == "Catalan"):
        x[26]=1    
    elif(lan == "Maltese"):
        x[27]=1 
    elif(lan == "Slovenian"):
        x[28]=1    
    elif(lan == "Indonesian"):
        x[29]=1    
    elif(lan == "Dutch"):
        x[30]=1
    elif(lan == "Chinese_Taiwan"):
        x[31]=1 
    elif(lan == "Sakha"):
        x[32]=1 
    elif(lan == "Polish"):
        x[33]=1 
    elif(lan == "Czech"):
        x[34]=1 
    elif(lan == "Romanian"):
        x[35]=1 
    elif(lan == "Mangolian"):
        x[36]=1 
    elif(lan == "Italian"):
        x[37]=1 
    elif(lan == "Chinese_Hongkong"):
        x[38]=1 
    elif(lan == "Estonian"):
        x[39]=1 
    elif(lan == "Basque"):
        x[40]=1 
    elif(lan == "Esperanto"):
        x[41]=1 
    elif(lan == "Frisian"):
        x[42]=1 
    elif(lan == "Turkish"):
        x[43]=1 
    elif(lan == "Chuvash"):
        x[44]=1 
    else:
        x = x
    return x       

In [15]:
#TODO add age
def process_input(ethnicity,gender,language,speaker_emb,audio_emb):
    speaker_emb = np.asarray(speaker_emb, dtype=np.float32)
    speaker_emb = speaker_emb.squeeze()
    audio_emb = np.asarray(audio_emb, dtype=np.float32)
    audio_emb = audio_emb.squeeze()
    #print(speaker_emb.shape)
    h = np.vstack((ethnicity, gender))
    h = np.vstack((h, language))
    h = np.vstack((h, speaker_emb))
    h = np.vstack((h, audio_emb))

    j = np.zeros(768)
    j = np.tile(j,(256-h.shape[0],1))
    h = np.vstack((h, j))
    #print(h.shape)
    return h.tolist()

from PIL import Image
def process_image_path(path):
    
    return path

In [16]:
for index, row in data_frame.iterrows():
    x = speaker_emb_preprocess(data_frame.loc[index,"SPEAKER_EMB"])
    x = [x]
    data_frame.loc[index,"SPEAKER_EMB"] = x

#CHECK
for index, row in data_frame.iterrows():
    x = audio_emb_preprocess2(data_frame.loc[index,"AUDIO_EMB"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"AUDIO_EMB"] = x

for index, row in data_frame.iterrows():
    x = process_gender(data_frame.loc[index,"caption_g"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"caption_g"] = x

for index, row in data_frame.iterrows():
    x = process_language(data_frame.loc[index,"caption_l"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"caption_l"] = x
    
for index, row in data_frame.iterrows():
    x = process_ethnicity(data_frame.loc[index,"caption_e"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"caption_e"] = x

data_frame['INPUT'] = data_frame['SPEAKER_EMB']

for index, row in data_frame.iterrows():
    x = process_input(data_frame.loc[index,"caption_e"],data_frame.loc[index,"caption_g"],
                    data_frame.loc[index,"caption_l"],data_frame.loc[index,"SPEAKER_EMB"],
                    data_frame.loc[index,"AUDIO_EMB"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"INPUT"] = x

data_frame = data_frame.drop(['caption_e', 'caption_g','caption_l','SPEAKER_EMB','AUDIO_EMB'], axis=1)

for index, row in data_frame.iterrows():
    x = process_image_path(data_frame.loc[index,"image_path"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"image_path"] = x

input = data_frame['INPUT'].to_numpy()
input = np.array([np.array(xi) for xi in input])




output = data_frame['image_path'].to_numpy()
output = np.array([np.array(xi) for xi in output])
output.squeeze().shape

(0,)

In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader

input = torch.from_numpy(input)
input = input.to(torch.float)

output = torch.from_numpy(output)
output = output.to(torch.float)

input = input.squeeze()
output = output.squeeze()

#my_dataset = TensorDataset(output,input) # create your datset
#my_dataloader = DataLoader(my_dataset) # create your dataloader

In [19]:
input.shape

torch.Size([256, 768])

In [18]:
import torch
from imagen_pytorch import Unet, Imagen, ImagenTrainer
from imagen_pytorch.data import Dataset

unet1 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
    layer_cross_attns = (False, True, True, True)
)

unet2 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = (2, 4, 8, 8),
    layer_attns = (False, False, False, True),
    layer_cross_attns = (False, False, False, True)
)

#unet = Unet(
#    dim = 32,
#    dim_mults = (1, 2, 4, 8),
#    num_resnet_blocks = 1,
#    layer_attns = (False, False, False, True),
#    layer_cross_attns = False
#)

# imagen, which contains the unet above

#imagen = Imagen(
#    unets = unet,
#    image_sizes = 32,
#    timesteps = 1000
#)

imagen = Imagen(
    unets = (unet1, unet2),
    image_sizes = (image_size, image_size),
    timesteps = 1000,
    cond_drop_prob = 0.1
).cuda()

trainer = ImagenTrainer(
    imagen = imagen,
    split_valid_from_train = True # whether to split the validation dataset from the training
).cuda()

print('Using model file ' + model_filename)
trainer.load(model_filename)

images = trainer.sample(text_embeds=input,stop_at_unet_number=2,batch_size = 1, return_pil_images = True) # returns List[Image]
images[0].save('test.png')



The base dimension of your u-net should ideally be no smaller than 128, as recommended by a professional DDPM trainer https://nonint.com/2022/05/04/friends-dont-let-friends-train-small-diffusion-models/
Using model file imagen_two_unets_24s.pt
checkpoint loaded from imagen_two_unets_24s.pt


0it [00:00, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

IndexError: too many indices for tensor of dimension 1

In [ ]:
input

In [ ]:
import os
os._exit(0)